In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

from tensorflow import keras
from tensorflow.keras import layers

import lightgbm as lgb

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv',index_col=0)
test = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv',index_col=0)
submission = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
data = pd.concat([train, test], axis=0).copy()

x = data.drop('target', axis=1)
x_train = x.loc[train.index]
x_test = x.loc[test.index]
y_train = train['target']

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, random_state=42)

In [ ]:
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid)

num_round = 100
param = {'num_leaves': 31, 'objective': 'regression', 'metric':'rsme'}
model = lgb.train(param, train_data, num_round, valid_sets=[valid_data])

In [ ]:
# # Input shape & random state
# tf.random.set_seed(0)
# _, input_shape = x_train.shape

# def root_mean_squared_error(y_true, y_pred):
#     return tf.math.sqrt(tf.math.reduce_mean(tf.math.squared_difference(y_true, y_pred)))


# # Build model
# model = keras.Sequential([
#                           keras.Input(shape=(input_shape,)),
#                           layers.Dense(128, activation='relu'),
#                           layers.Dense(128, activation='relu'),
#                           layers.Dense(128, activation='relu'),
#                           layers.Dense(1),
#                           ])

# # Compile model
# model.compile(
#              optimizer="adam",
#              loss=root_mean_squared_error,
#              metrics=["mean_squared_error"],
#              )

# # Set early stopping
# early_stop = [keras.callbacks.EarlyStopping(
#                     monitor="val_loss",
#                     min_delta=1e-2,
#                     patience=3,
#                     verbose=1,
#                     )]
    
# # Train model
# history = model.fit(
#                     x_train,
#                     y_train,
#                     batch_size=64,
#                     epochs=30,
#                     callbacks=early_stop,
#                     validation_split=0.2,
#                     )

In [ ]:
# model = SGDRegressor(max_iter=30000, random_state=42)

# score = cross_val_score(model, x_train, y_train, scoring='neg_root_mean_squared_error')
# score = abs(score.mean())
# score

In [ ]:
# model = SGDRegressor(random_state=42)
# model.fit(x_train, y_train)
# preds = model.predict(x_valid)
# score = mean_squared_error(y_valid, preds, squared=False)
# score

# model = SGDRegressor(random_state=42)
# model.fit(x_train, y_train)

In [ ]:
preds = model.predict(x_test)
submission['target'] = preds
submission.to_csv("submission.csv", index=False)